# Deploying Lilac to a HuggingFace Space

This notebook will show you how to deploy a project directory, or a single dataset to HuggingFace spaces.

This will allow you to make a public or private space that hosts your datasets so you can share with others.

Example: https://huggingface.co/spaces/lilacai/lilac

The accompanying guide can be found [here](https://lilacml.com/huggingface/huggingface_spaces.html).

All of these scripts require:

- You to be logged in with HuggingFace with `huggingface-cli login`.
- Have the `huggingface_hub` pip package installed.


## Deploying a single dataset from a config

We can deploy a single dataset to a HuggingFace space with a few lines of Python.

The dataset will get loaded from the HuggingFace space, so you don't have to load it locally!

If you have already loaded a project locally and want to deploy it, skip to the next section.


In [1]:
from IPython.display import display
import lilac as ll

/Users/nikhil/Code/lilac/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[`deploy_config`](https://lilacml.com/api_reference/index.html#lilac.deploy_config) will deploy a Lilac instance from a Lilac configuration object. The lilac config object below will tell the HuggingFace space to load the dataset. It will also compute the gte-small embedding and the `text_statistics` signal when it boots up on the space.

You can also choose to compute embeddings, and signals on the HuggingFace space upon bootup of the space. See [lilac.Config](https://lilacml.com/api_reference/index.html#lilac.Config) for reference, or the guide on [Lilac Projects](https://lilacml.com/projects/projects.html).

NOTE: The `deploy_config` script will always create a private space. To make it public, go to the HuggingFace UI, click Settings, and click "Make public" under "Change space visibility."


In [2]:
ll.deploy_config(
  hf_space='nsthorat-lilac/nikhil-demo',
  # Create the space if it doesn't exist.
  create_space=True,
  config=ll.Config(datasets=[
    ll.DatasetConfig(
      namespace='local',
      name='glue_ax',
      source=ll.HuggingFaceSource(dataset_name='glue', config_name='ax'),
      # NOTE: Remove embeddings and signals if you just want to visualize the dataset without any
      # enrichments.
      embeddings=[
        # Compute gte-small over 'hypothesis'.
        ll.EmbeddingConfig(path='hypothesis', embedding='gte-small'),
      ],
      signals=[ll.SignalConfig(path='hypothesis', signal=ll.TextStatisticsSignal())])
  ]),
  # No persistent storage for HuggingFace. If you want to use persistent storage,
  # set this to 'small', 'medium', or 'large'.
  # NOTE: Persistent storage is not free. See https://huggingface.co/docs/hub/spaces-storage
  storage=None)


Creating huggingface space https://huggingface.co/spaces/nsthorat-lilac/nikhil-demo
The space will be created as private. You can change this from the UI.
Created: https://huggingface.co/spaces/nsthorat-lilac/nikhil-demo
Deploying project: /var/folders/35/q7dnsp_x3v3b8mp46bb9w0qr0000gn/T/tmpfeogfbkz

Copying root files...

Uploading concepts:  []

Uploading cache files: []

Done! View your space at https://huggingface.co/spaces/nsthorat-lilac/nikhil-demo


## Deploying a project

If we've already loaded some datasets and embeddings locally, we can upload the entire project (or a subset) to a space and re-use local computation using
[`deploy_project`](https://lilacml.com/api_reference/index.html#lilac.deploy_project).

Loading data will not happen on the space automatically on bootup.

NOTE: The `deploy_project` script will always create a private space. To make it public, go to the HuggingFace UI, click Settings, and click "Make public" under "Change space visibility."


In [3]:
ll.deploy_project(
  hf_space='nsthorat-lilac/nikhil-project-demo',
  project_dir='./data',
  datasets=['local/glue_ax'],  # This is optional. If not defined, uploads all datasets.
  # Create the space if it doesn't exist.
  create_space=True,
  # The hf_token is required unless you set make_datasets_public=True. By default, uploaded datasets
  # are made private.
  # See: https://huggingface.co/docs/hub/security-tokens
  hf_token='your_huggingface_read_token')

Creating huggingface space https://huggingface.co/spaces/nsthorat-lilac/nikhil-project-demo
The space will be created as private. You can change this from the UI.
Created: https://huggingface.co/spaces/nsthorat-lilac/nikhil-project-demo
Deploying project: ./data

Copying root files...

Uploading datasets:  ['local/glue_ax']
Uploading "local/glue_ax" to HuggingFace dataset repo https://huggingface.co/datasets/nsthorat-lilac/nikhil-project-demo-local-glue_ax



data-00000-of-00001.parquet:   0%|          | 0.00/116k [00:00<?, ?B/s]










data-00000-of-00001.parquet:   7%|▋         | 8.19k/116k [00:00<00:01, 67.6kB/s]






spans.pkl: 100%|██████████| 53.0k/53.0k [00:00<00:00, 165kB/s] 
hnsw.lookup.pkl: 100%|██████████| 53.4k/53.4k [00:00<00:00, 149kB/s] 
data-00000-of-00001.parquet: 100%|██████████| 46.8k/46.8k [00:00<00:00, 123kB/s] 
data-00000-of-00001.parquet: 100%|██████████| 116k/116k [00:00<00:00, 283kB/s]  
data-00000-of-00001.parquet:   0%|          | 0.00/939 [00:00<?, ?B/s]



data-00000-of-00001.parquet: 100%|██████████| 939/939 [00:00<00:00, 16.8kB/s]


data-00000-of-00001.parquet: 100%|██████████| 47.4k/47.4k [00:00<00:00, 435kB/s]
data-00000-of-00001.parquet: 100%|██████████| 43.5k/43.5k [00:00<00:00, 227kB/s]
data-00000-of-00001.parquet: 100%|██████████| 41.2k/41.2k [00:00<00:00, 196kB/s]
hnsw.hnswlib.bin: 100%|██████████| 1.86M/1.86M [00:01<00:00, 1.25MB/s]


Upload 9 LFS files: 100%|██████████| 9/9 [00:01<00:00,  5.55it/s

Uploading concepts:  ['local/aliens', '100712716653593140239/aliens', '100712716653593140239/private_aliens']

Uploading cache files: ['concept/local/aliens/gte-small.pkl', 'concept/100712716653593140239/aliens/gte-small.pkl', 'concept/100712716653593140239/private_aliens/gte-small.pkl']



Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]


gte-small.pkl: 100%|██████████| 21.8k/21.8k [00:00<00:00, 266kB/s]
gte-small.pkl: 100%|██████████| 10.8k/10.8k [00:00<00:00, 111kB/s]
gte-small.pkl: 100%|██████████| 28.4k/28.4k [00:00<00:00, 231kB/s]
Upload 3 LFS files: 100%|██████████| 3/3 [00:00<00:00,  8.47it/s]


Done! View your space at https://huggingface.co/spaces/nsthorat-lilac/nikhil-project-demo
